Import packages

In [1]:
import math
import os.path
import pickle

import numpy as np
import timm
import torch
from matplotlib import pyplot as plt
from timm.loss import LabelSmoothingCrossEntropy
from torch.optim import lr_scheduler
from tqdm import tqdm

from torch import nn
from torch.utils.data import DataLoader
from torchsummary import summary
import torch.nn.functional as F
from Models.ResNet import ResNet
from Models.ViT_pretrained.pytorch_pretrained_vit import ViT
# from Models.ViT import ViT
# from Models.ViT_luci import ViT_luci
# from pytorch_pretrained_vit import ViT

from datasets import ODIRDataset, get_ODIRDataset, get_test_dataset
from params import *
from utils import train
import torchvision.models as models


training parameters

In [2]:
train_ratio = 0.85
batch_size = 8
epochs = 2
lr = 1e-2
normalize_mean_resnet = [0.485, 0.456, 0.406]
normalize_std_resnet = [0.229, 0.224, 0.225]
data_root = 'C:/Users/S-Vec/Dropbox/ExpertViT/Datasets/ODIR'
normalize_mean_ViTPretrained = [0.5, 0.5, 0.5]
normalize_std_ViTPretrained = [0.5, 0.5, 0.5]

random_seed =42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

Create dataset

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

# train_data_loader, val_data_loader, train_size, val_size, input_shape, num_classes = get_ODIRDataset(data_root, train_ratio, batch_size, normalize_using_dataset_values=True)
train_data_loader, val_data_loader, train_size, val_size, input_shape, num_classes = get_ODIRDataset(data_root, train_ratio, batch_size, False, normalize_mean_ViTPretrained, normalize_std_ViTPretrained)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/S-Vec/Dropbox/ExpertViT/Datasets/ODIR\\full_df.csv'

Set up Model

In [ ]:
model = timm.create_model('inception_v4', pretrained=True, num_classes=num_classes).to(device)

print("Model Summary: ")
summary(model, input_size=input_shape)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, nesterov=True, lr=0.01, weight_decay=0.)  # parameter used in https://github.com/Zoe0123/Vision-Transformer-for-Chest-X-Ray-Classification/blob/main/vit.ipynb

Training the model

In [ ]:
training_history = train(model, optimizer, criterion, train_data_loader, val_data_loader)


Plot training history

In [ ]:
plt.plot(training_history['loss_train'], label='Train loss')
plt.plot(training_history['loss_val'], label='Val loss')
plt.legend()
plt.title("Losses for ViT")
plt.show()

plt.plot(training_history['acc_train'], label='Train accuracy')
plt.plot(training_history['acc_val'], label='Val accuracy')
plt.legend()
plt.title("Accuracies for ViT")
plt.show()

Visualize activation map